In [4]:
import sys
import os
import random
import string
import shlex 
import time 
import tempfile 
import shutil 
import subprocess

# --- 0. LIBRARY NOTE ---
# In Binder, libraries are installed via requirements.txt before the notebook starts.
# We skip manual pip installation here to avoid permissions errors and save time.
print("Binder Environment detected: Libraries are pre-installed via requirements.txt.")

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import ipywidgets as widgets
from IPython.display import display, clear_output
import requests 

import stem.process
import stem.control 

# Global variables for Tor process and directory
tor_process = None
tor_data_dir = None
control_password = None

# --- 1. WebDriver Setup (Binder Optimized) ---
def get_binder_driver(proxy=None):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev_shm_usage')
    
    # This path matches the Dockerfile installation
    chrome_options.binary_location = '/usr/bin/google-chrome'

    if proxy:
        chrome_options.add_argument(f'--proxy-server={proxy}')
        print(f"Configuring WebDriver with proxy: {proxy}")

    try:
        # In Docker, we let Selenium find the driver automatically
        return webdriver.Chrome(options=chrome_options)
    except Exception as e:
        print(f"Error starting Chrome: {e}")
        return None

driver = get_binder_driver()
if driver:
    print("Chrome WebDriver installed and initialized successfully in headless mode.")

# --- 2. URL Input and Navigation ---

url_input = widgets.Text(
    value='https://faucetearner.org/?r=508070998956',
    placeholder='Type a URL',
    description='Enter URL:',
    disabled=False
)

def navigate_to_url(url):
    print(f"Navigating to: {url}")
    driver.get(url)

navigate_button = widgets.Button(description='Go to URL')
def on_navigate_button_click(b):
    navigate_to_url(url_input.value)
navigate_button.on_click(on_navigate_button_click)

# --- 3. Click 'Get Started' Button ---

def click_get_started_button():
    try:
        get_started_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.hero-btn.primary'))
        )
        get_started_button.click()
        print("Successfully clicked the 'Get Started' button.")
    except Exception as e:
        print(f"Error clicking 'Get Started' button: {e}")

get_started_widget = widgets.Button(description='Click Get Started')
def on_get_started_button_click(b):
    click_get_started_button()
get_started_widget.on_click(on_get_started_button_click)

# --- 4. Registration Form Inputs and Auto-Generation ---

def generate_random_string(length=10, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.choice(chars) for i in range(length))

username_input = widgets.Text(
    value='', placeholder='Enter Username', description='Username:', disabled=False
)
email_input = widgets.Text(
    value='', placeholder='Enter Email', description='Email:', disabled=False
)
password_input = widgets.Text(
    value='', placeholder='Enter Password', description='Password:', disabled=False
)
confirm_password_input = widgets.Text(
    value='', placeholder='Confirm Password', description='Confirm Password:', disabled=False
)

def generate_username(b):
    username_input.value = generate_random_string(length=8)
def generate_email(b):
    email_prefix = generate_random_string(length=8, chars=string.ascii_lowercase + string.digits)
    email_input.value = f"{email_prefix}@gmail.com"
def generate_password(b):
    all_chars = string.ascii_letters + string.digits + string.punctuation
    password_input.value = generate_random_string(length=12, chars=all_chars)
def confirm_password(b):
    confirm_password_input.value = password_input.value

gen_username_button = widgets.Button(description='Generate Username')
gen_email_button = widgets.Button(description='Generate Email')
gen_password_button = widgets.Button(description='Generate Password')
gen_confirm_password_button = widgets.Button(description='Confirm Password')

gen_username_button.on_click(generate_username)
gen_email_button.on_click(generate_email)
gen_password_button.on_click(generate_password)
gen_confirm_password_button.on_click(confirm_password)

# --- 5. Fill Registration Fields ---

def fill_registration_fields(username, email, password, confirm_password):
    try:
        username_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'username')))
        username_field.send_keys(username)

        email_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'email')))
        email_field.send_keys(email)

        password_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'password')))
        password_field.send_keys(password)

        confirm_password_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'confirmPassword')))
        confirm_password_field.send_keys(confirm_password)

        print("Successfully filled registration fields using By.ID.")
    except Exception as e:
        print(f"Error filling registration fields: {e}")

fill_form_button = widgets.Button(description='Fill Registration Fields')
def on_fill_form_button_click(b):
    fill_registration_fields(username_input.value, email_input.value, password_input.value, confirm_password_input.value)
fill_form_button.on_click(on_fill_form_button_click)

# --- 6. Click Checkbox ---

def click_checkbox():
    try:
        checkbox = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@type='checkbox']"))
        )
        driver.execute_script("arguments[0].click();", checkbox)
        print("Successfully clicked the 'Terms and Conditions' checkbox via JavaScript.")
    except Exception as e:
        print(f"Error clicking checkbox: {e}")

click_checkbox_button = widgets.Button(description='Click Terms Checkbox')
def on_click_checkbox_button(b):
    click_checkbox()
click_checkbox_button.on_click(on_click_checkbox_button)

# --- 7. Submit Registration Form ---

def submit_registration_form():
    try:
        submit_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'registerButton'))
        )
        driver.execute_script("arguments[0].click();", submit_button)
        print("Successfully clicked the 'Submit Registration' button via JavaScript.")
    except Exception as e:
        print(f"Error submitting registration form: {e}")

submit_form_button = widgets.Button(description='Submit Registration')
def on_submit_form_button_click(b):
    submit_registration_form()
submit_form_button.on_click(on_submit_form_button_click)

# --- 8. Capture Registration Output ---

def capture_output():
    try:
        print("Attempting to capture registration output...")
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        print(f"Current URL after submission: {driver.current_url}")
        print(f"Page Title: {driver.title}")
    except Exception as e:
        print(f"Error capturing registration output: {e}")

capture_result_button = widgets.Button(description='Capture Registration Result')
def on_capture_result_button_click(b):
    capture_output()
capture_result_button.on_click(on_capture_result_button_click)

# --- 9. IP Display Functionality ---

def get_public_ip():
    try:
        response = requests.get('https://api.ipify.org')
        print(f"Current public IP address: {response.text.strip()}")
    except Exception as e:
        print(f"Error fetching public IP address: {e}")

ip_button = widgets.Button(description='Get Public IP')
def on_ip_button_click(b):
    get_public_ip()
ip_button.on_click(on_ip_button_click)

# --- 10. Proxy Configuration (Refined for Binder) ---

proxy_ip_input = widgets.Text(placeholder='e.g., 192.168.1.1', description='Proxy IP:')
proxy_port_input = widgets.Text(placeholder='e.g., 8080', description='Proxy Port:')

def configure_driver_with_proxy(proxy_ip=None, proxy_port=None):
    global driver
    if driver:
        driver.quit()
    
    proxy_address = None
    if proxy_ip and proxy_port:
        proxy_address = f"{proxy_ip}:{proxy_port}"
    
    driver = get_binder_driver(proxy=proxy_address)

apply_proxy_button = widgets.Button(description='Apply Proxy / Reconfigure')
def on_apply_proxy_button_click(b):
    configure_driver_with_proxy(proxy_ip_input.value.strip(), proxy_port_input.value.strip())
apply_proxy_button.on_click(on_apply_proxy_button_click)

# --- 11. Tor Integration (Binder Compatible) ---

def cleanup_tor():
    global tor_process, tor_data_dir
    if tor_process:
        try:
            tor_process.kill()
        except: pass
    if tor_data_dir and os.path.exists(tor_data_dir):
        try:
            shutil.rmtree(tor_data_dir)
        except: pass

def configure_driver_with_tor_proxy():
    global driver, tor_process, tor_data_dir, control_password
    
    # 1. Start Tor
    cleanup_tor()
    control_password = generate_random_string(16)
    
    # Generate hash using system tor
    hashed = os.popen(f"tor --hash-password {shlex.quote(control_password)}").read().strip().split('\n')[-1]
    
    tor_data_dir = tempfile.mkdtemp()
    print("Starting Tor daemon...")
    
    try:
        tor_process = stem.process.launch_tor_with_config(
            config={
                'SocksPort': '9050',
                'ControlPort': '9051',
                'HashedControlPassword': hashed,
                'DataDirectory': tor_data_dir
            },
            timeout=120
        )
        print("Tor started. Configuring driver...")
        
        # 2. Configure Driver
        global driver
        if driver: driver.quit()
        
        # SOCKS5 proxy for Tor
        driver = get_binder_driver(proxy="socks5://127.0.0.1:9050")
        print("Driver configured with Tor.")
        
    except Exception as e:
        print(f"Error starting Tor: {e}")

apply_tor_proxy_button = widgets.Button(description='Apply Tor Proxy')
apply_tor_proxy_button.on_click(lambda b: configure_driver_with_tor_proxy())

def renew_tor_circuit():
    try:
        with stem.control.Controller.from_port(port=9051) as controller:
            controller.authenticate(password=control_password)
            controller.signal(stem.Signal.NEWNYM)
            print("Tor circuit renewed (New IP requested).")
    except Exception as e:
        print(f"Error renewing Tor: {e}")

renew_tor_ip_button = widgets.Button(description='Renew Tor IP')
renew_tor_ip_button.on_click(lambda b: renew_tor_circuit())

# --- 12. Automated Loop Functionality ---

loop_count_input = widgets.IntText(value=1, description='Number of Loops:', min=1)

def automate_all_steps_in_loop(num_loops):
    for i in range(num_loops):
        # Syntax Error Fixed: Removed trailing backslash
        print(f"\n--- Starting automation loop {i+1}/{num_loops} ---")
        
        # 1. Apply Tor (Optional, based on your logic)
        print("Step 1: Renewing Tor IP...")
        renew_tor_circuit() 
        time.sleep(5) 

        # 2. Go to URL
        print("Step 2: Navigating to URL...")
        navigate_to_url(url_input.value)
        time.sleep(3)

        # 3. Click 'Get Started'
        print("Step 3: Clicking 'Get Started'...")
        click_get_started_button()
        time.sleep(3)

        # 4. Generate & Fill
        print("Step 4: Filling forms...")
        generate_username(None)
        generate_email(None)
        generate_password(None)
        confirm_password(None)
        fill_registration_fields(username_input.value, email_input.value, password_input.value, confirm_password_input.value)
        time.sleep(3)

        # 5. Checkbox
        print("Step 5: Clicking Checkbox...")
        click_checkbox()
        time.sleep(2)

        # 6. Submit
        print("Step 6: Submitting...")
        submit_registration_form()
        time.sleep(5)

        # 7. Capture
        print("Step 7: Capturing output...")
        capture_output()
        
        # 8. IP Check
        get_public_ip()
        
        print(f"--- Finished loop {i+1} ---")

automate_button = widgets.Button(description='Automate All Steps (Looped)')
def on_automate_button_click(b):
    automate_all_steps_in_loop(loop_count_input.value)
automate_button.on_click(on_automate_button_click)

# --- 13. Display All Widgets ---

username_row = widgets.HBox([username_input, gen_username_button])
email_row = widgets.HBox([email_input, gen_email_button])
password_row = widgets.HBox([password_input, gen_password_button])
confirm_password_row = widgets.HBox([confirm_password_input, gen_confirm_password_button])

proxy_ip_row = widgets.HBox([proxy_ip_input])
proxy_port_row = widgets.HBox([proxy_port_input])

all_widgets_display = widgets.VBox([
    widgets.HBox([url_input, navigate_button]),
    get_started_widget,
    username_row,
    email_row,
    password_row,
    confirm_password_row,
    fill_form_button,
    click_checkbox_button,
    submit_form_button,
    capture_result_button,
    ip_button,
    proxy_ip_row,
    proxy_port_row,
    apply_proxy_button,
    apply_tor_proxy_button,
    renew_tor_ip_button,
    widgets.HBox([loop_count_input, automate_button])
])

display(all_widgets_display)
print("All interactive control widgets displayed.")

Binder Environment detected: Libraries are pre-installed via requirements.txt.
Chrome WebDriver installed and initialized successfully in headless mode.


All interactive control widgets displayed.
